ARE YOU READY! 👏  I'm actually quite excited as this is a project as I have been wanting to do for a while...  Evolutionary Search over Deep Neural Network Architectures 🐣.  This is actually an old concept [1](https://ieeexplore.ieee.org/document/784219/) that has had a bit of a renaissance [2](https://arxiv.org/pdf/1703.00548.pdf) 📄.  Cool packages such as [DEVOL](https://github.com/joeddav/devol) implement a nice version of this for 2D Convoltuion, but I really wanted to do this myself, in order to implement this for other problems like Feed Forward Neural Networks or 1d Convolution 🤓.  [DEAP](http://deap.readthedocs.io/en/master/api/tools.html) is a popular python library for evolutionary computing and seems really versatile, so check it out.  
  
Ok, time to import some libraries...*boring*📚.  

In [1]:
import random
import numpy as np

from deap import base, tools, creator

import tensorflow as tf
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers

from sklearn.datasets import load_boston

import pandas as pd

Using TensorFlow backend.


We are going try the classic boston houing price dataset.  🍳

In [3]:
data = load_boston()

So the Genetic Algorithms involves four main steps...  
  
__Initialize/Selection:__  Where we select new genes.  
__Evalutation:__  Where we evaluate these genes.  
__Cross-over:__  Where we 'breed' out individuals to exchange genes.  
__Mutation:__  Where we introduce new genetic material.  
  
Here, we let the number of neurons and the number of layers in our network be our genes and our loss function is the loss on our hold-out set.  

In [4]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

As you can see here, regulatization is really important or the network can get too big, so I included regularization on each layer's regularization for backprop and then added a regularization term for Genetic Algorithm.  

In [11]:
def evalOneMax(individual, alpha=1/50):
    inputs = Input(shape=(data.data.shape[1],))
    
    dense1 = Dense(abs(individual[0]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[0] != 0 else (inputs)
    
    dense2 = Dense(abs(individual[1]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[1] != 0 else (dense1)
    
    dense3 = Dense(abs(individual[2]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[2] != 0 else (dense2)
    
    output = Dense(1, kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='softmax')(inputs)
    
    model = Model(inputs=inputs, outputs=output)
    
    model.compile('adam', loss=tf.nn.log_poisson_loss)
    model.fit(data.data, data.target,
                epochs=5,
                batch_size=20,
                shuffle=True,
                validation_split=False,
                verbose=0
           )
    return  (model.evaluate(data.data, data.target, batch_size=128, verbose=0).tolist() + alpha*sum(individual),)

In [12]:
toolbox = base.Toolbox()

N_ATTR = 3

# Attribute generator 
toolbox.register("attr_bool", random.randint,0,64)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, N_ATTR)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#Evolutionary Strategies
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt,low=0, up=64, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=3)

TIME TO EVOLVE!

In [13]:
def main(N_POP=5, CXPB = 0.2, MUTPB = 0.5, GEN=10):
    """
    # N_POP is the population size
    # MUTPB is the probability for mutating an individual
    # CXPB  is the probability with which two individuals are crossed
    # GEN is the number of generations to be run by the algorithm
    """
    
    pop = toolbox.population(N_POP)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit    
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 5 and g < GEN:
        # A new generation
        g = g + 1
        print(f"-- Generation {g} --")
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
              
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
              
        pop[:] = offspring
              
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        print(f'Offspring: {offspring}')
        print(f'Fitness Funcitons: {list(map(lambda x: round(x, 3),fits))}')
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
    return pop[fits.index(min(fits))]

In [14]:
pop = main(N_POP=10, GEN=10)

W0807 11:53:07.238829 140093439174464 deprecation_wrapper.py:119] From /home/marcusskky/.conda/envs/Evolution/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 11:53:07.355266 140093439174464 deprecation_wrapper.py:119] From /home/marcusskky/.conda/envs/Evolution/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 11:53:07.376158 140093439174464 deprecation_wrapper.py:119] From /home/marcusskky/.conda/envs/Evolution/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 11:53:07.491070 140093439174464 deprecation_wrapper.py:119] From /home/marcusskky/.conda/envs/Evolution/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprec

-- Generation 1 --
Offspring: [[24, 2, 14], [22, 48, 45], [21, 32, 5], [21, 47, 40], [16, 48, 45], [24, 48, 45], [10, 26, 22], [22, 2, 37], [22, 2, 14], [24, 48, 45]]
Fitness Funcitons: [-18.515, -17.185, -18.359, -17.399, -17.288, -17.288, -18.439, -18.69, -18.69, -17.288]
  Min -18.690363441889463
  Max -17.185317512934386
  Avg -17.914449157503753
  Std 0.6333322211456874
-- Generation 2 --
Offspring: [[22, 2, 14], [22, 2, 14], [21, 32, 5], [27, 14, 5], [24, 2, 14], [22, 2, 14], [10, 29, 12], [10, 26, 22], [7, 47, 40], [41, 26, 22]]
Fitness Funcitons: [-18.69, -18.69, -18.359, -18.359, -18.515, -18.69, -18.439, -18.439, -17.399, -18.439]
  Min -18.690363441889463
  Max -17.399129767738312
  Avg -18.402245040893554
  Std 0.3571849237361432
-- Generation 3 --
Offspring: [[22, 2, 14], [22, 2, 14], [24, 2, 47], [22, 2, 14], [22, 22, 14], [35, 2, 14], [22, 2, 14], [22, 2, 14], [22, 2, 14], [10, 29, 12]]
Fitness Funcitons: [-18.69, -18.69, -17.979, -18.664, -18.69, -18.69, -18.69, -18.69,

This is awesome! Lets check it out.  What's really interesting is that this network has more neurons in the middle layer.  

In [15]:
print(pop)

[22, 28, 26]


In [16]:
loss = evalOneMax(pop, alpha=0)
print(loss)

(-19.369102990674406,)
